# Dev 04 - Spectral Processing

Using the `toshi` module to read in data and process for spectra

In [1]:
import os
import time
import numpy as np
import scipy.signal
import matplotlib
import matplotlib.pyplot as plt

import toshi

file = os.path.expanduser('~/Downloads/20200614_150015.256007-87-02.iqData.XXXX.AKITA.dat')
# file = os.path.expanduser('~/Downloads/20200615_010423.194421-8C-02.iqData.XXXX.AKITA.dat')
filesize = os.path.getsize(file)

In [2]:
s = time.time()
ray_pulses, cpi_headers = toshi.read(file)
e = time.time()
print('Data read in {:.2f} s'.format(e - s))

Data read in 1.99 s


In [3]:
naz = len(ray_pulses)
ngate = ray_pulses[0][0].ngate_long_hi
# ngate = ray_pulses[0][0].ngate_short_hi
z = np.zeros((naz, ngate), dtype=np.single)
v = np.zeros((naz, ngate), dtype=np.single)
e = np.zeros((naz), dtype=np.single)
a = np.zeros((naz), dtype=np.single)
r = np.zeros((ngate), dtype=np.single)

for k, (pulses, cpi_header) in enumerate(zip(ray_pulses, cpi_headers)):
    p = np.zeros((len(pulses), ngate), dtype=np.csingle)
    for j, pulse in enumerate(pulses):
        p[j, :] = pulse.h_long_hi * np.exp(-1j * pulse.phase_h_long)
    pp = p[1:, :] * np.conj(p[:-1, :])
    v[k, :] = np.angle(np.sum(pp, axis=0))
    z[k, :] = np.sum(np.abs(p), axis=0)

# Sweep operations
z = 20 * np.log10(z) - 60
# v[z < -10] = np.nan

In [4]:
# import struct

# pri_struct = struct.Struct('I'*32)
# n = pri_struct.unpack_from(ray_pulses[0][1].raw_pri)
# for k, x in enumerate(n):
#     print('{:2d}: {:08X}'.format(k, x))

In [5]:
w, h = 800, 450
dpi = 72
fig = plt.figure(figsize=[x / dpi for x in (w, h)], dpi=dpi)

<Figure size 800x450 with 0 Axes>